In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, when
import pandas as pd
from pyspark.sql.functions import monotonically_increasing_id

In [2]:
spark = SparkSession.builder.appName('MIGRACAO_HIVE').getOrCreate()

23/11/26 18:31:59 WARN Utils: Your hostname, rodrigo-Nitro-AN515-54 resolves to a loopback address: 127.0.1.1; using 192.168.0.112 instead (on interface wlp8s0)
23/11/26 18:31:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/11/26 18:32:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_total_votos_candidato = spark \
    .read \
    .parquet('../data/processed/total_votos_candidato.parquet')
df_total_votos_candidato.show()

+-----------+--------+------------+------------+--------------------+--------------------+----------------+----------------+----------------------------+------------------+-----------------------+
|ANO_ELEICAO|CD_CARGO|CD_MUNICIPIO|NR_CANDIDATO|        NM_CANDIDATO|   NM_URNA_CANDIDATO|CD_SIT_TOT_TURNO|DS_SIT_TOT_TURNO|TOTAL_VOTOS_CANDIDATO_CIDADE|TOTAL_VOTOS_CIDADE|PERCENTUAL_VOTOS_CIDADE|
+-----------+--------+------------+------------+--------------------+--------------------+----------------+----------------+----------------------------+------------------+-----------------------+
|       2018|       6|       61018|        1720|EDUARDO NANTES BO...|   EDUARDO BOLSONARO|               2|   ELEITO POR QP|                        2179|             16124|                  13.51|
|       2018|       6|       61018|        1011|ALEXANDRE ARAUJO ...|ALEXANDRE DAUAGE ...|               5|        SUPLENTE|                           0|             16124|                    0.0|
|       2018|  

In [4]:
df_total_votos_candidato.count()

4382775

In [5]:
df_total_votos_candidato = df_total_votos_candidato \
    .select(
        'NR_CANDIDATO',
        'NM_CANDIDATO',
        'NM_URNA_CANDIDATO',
        'DS_SIT_TOT_TURNO',
        'TOTAL_VOTOS_CANDIDATO_CIDADE',
        'TOTAL_VOTOS_CIDADE',
        'PERCENTUAL_VOTOS_CIDADE',
        'ANO_ELEICAO',
        'CD_CARGO',
        'CD_MUNICIPIO',
        'CD_SIT_TOT_TURNO'

    )

df_total_votos_candidato.show()

+------------+--------------------+--------------------+----------------+----------------------------+------------------+-----------------------+-----------+--------+------------+----------------+
|NR_CANDIDATO|        NM_CANDIDATO|   NM_URNA_CANDIDATO|DS_SIT_TOT_TURNO|TOTAL_VOTOS_CANDIDATO_CIDADE|TOTAL_VOTOS_CIDADE|PERCENTUAL_VOTOS_CIDADE|ANO_ELEICAO|CD_CARGO|CD_MUNICIPIO|CD_SIT_TOT_TURNO|
+------------+--------------------+--------------------+----------------+----------------------------+------------------+-----------------------+-----------+--------+------------+----------------+
|        1720|EDUARDO NANTES BO...|   EDUARDO BOLSONARO|   ELEITO POR QP|                        2179|             16124|                  13.51|       2018|       6|       61018|               2|
|        1011|ALEXANDRE ARAUJO ...|ALEXANDRE DAUAGE ...|        SUPLENTE|                           0|             16124|                    0.0|       2018|       6|       61018|               5|
|        1771|J

In [6]:
lista_colunas = df_total_votos_candidato.columns

In [7]:
lista_colunas

['NR_CANDIDATO',
 'NM_CANDIDATO',
 'NM_URNA_CANDIDATO',
 'DS_SIT_TOT_TURNO',
 'TOTAL_VOTOS_CANDIDATO_CIDADE',
 'TOTAL_VOTOS_CIDADE',
 'PERCENTUAL_VOTOS_CIDADE',
 'ANO_ELEICAO',
 'CD_CARGO',
 'CD_MUNICIPIO',
 'CD_SIT_TOT_TURNO']

In [8]:
list_cidade = [ linha['CD_MUNICIPIO'] for linha in df_total_votos_candidato.select('CD_MUNICIPIO').distinct().collect() ]
list_cidade

[71510,
 68098,
 61344,
 66613,
 67857,
 64874,
 61859,
 65013,
 71374,
 61786,
 63835,
 61654,
 64459,
 67911,
 65196,
 69515,
 65110,
 61158,
 61328,
 64033,
 66338,
 67474,
 65714,
 64254,
 61212,
 62456,
 61514,
 61077,
 62073,
 70912,
 71110,
 61298,
 71978,
 61271,
 62774,
 70718,
 61905,
 63037,
 68497,
 66257,
 67334,
 67156,
 69736,
 71650,
 67997,
 64610,
 62375,
 64335,
 66494,
 70394,
 71099,
 71218,
 63096,
 67490,
 67431,
 63673,
 63690,
 63711,
 70939,
 70190,
 63193,
 68411,
 63452,
 63797,
 66095,
 65072,
 70114,
 68470,
 71250,
 63339,
 70971,
 70556,
 66915,
 61417,
 62421,
 72451,
 71390,
 69159,
 64971,
 68594,
 64297,
 64998,
 67393,
 68217,
 63770,
 61778,
 63479,
 63258,
 68373,
 65579,
 61565,
 68950,
 61824,
 70599,
 64394,
 61131,
 64114,
 68535,
 64513,
 64530,
 63398,
 61697,
 69310,
 62790,
 63614,
 61662,
 65790,
 68659,
 64190,
 66753,
 64572,
 61140,
 67415,
 61093,
 70998,
 63371,
 62448,
 71838,
 66850,
 62596,
 64173,
 65455,
 65773,
 70955,
 69299,


In [9]:
df_total_votos_candidato = df_total_votos_candidato.withColumn('ID_LINHA', monotonically_increasing_id()) 

In [10]:
df_total_votos_candidato.show()

+------------+--------------------+--------------------+----------------+----------------------------+------------------+-----------------------+-----------+--------+------------+----------------+--------+
|NR_CANDIDATO|        NM_CANDIDATO|   NM_URNA_CANDIDATO|DS_SIT_TOT_TURNO|TOTAL_VOTOS_CANDIDATO_CIDADE|TOTAL_VOTOS_CIDADE|PERCENTUAL_VOTOS_CIDADE|ANO_ELEICAO|CD_CARGO|CD_MUNICIPIO|CD_SIT_TOT_TURNO|ID_LINHA|
+------------+--------------------+--------------------+----------------+----------------------------+------------------+-----------------------+-----------+--------+------------+----------------+--------+
|        1720|EDUARDO NANTES BO...|   EDUARDO BOLSONARO|   ELEITO POR QP|                        2179|             16124|                  13.51|       2018|       6|       61018|               2|       0|
|        1011|ALEXANDRE ARAUJO ...|ALEXANDRE DAUAGE ...|        SUPLENTE|                           0|             16124|                    0.0|       2018|       6|       610

In [11]:
numero_linha_particao = 1000
df_total_votos_candidato = df_total_votos_candidato.withColumn('PARTICAO_ID', (df_total_votos_candidato['ID_LINHA'] / numero_linha_particao).cast('integer'))
df_total_votos_candidato.show()

+------------+--------------------+--------------------+----------------+----------------------------+------------------+-----------------------+-----------+--------+------------+----------------+--------+-----------+
|NR_CANDIDATO|        NM_CANDIDATO|   NM_URNA_CANDIDATO|DS_SIT_TOT_TURNO|TOTAL_VOTOS_CANDIDATO_CIDADE|TOTAL_VOTOS_CIDADE|PERCENTUAL_VOTOS_CIDADE|ANO_ELEICAO|CD_CARGO|CD_MUNICIPIO|CD_SIT_TOT_TURNO|ID_LINHA|PARTICAO_ID|
+------------+--------------------+--------------------+----------------+----------------------------+------------------+-----------------------+-----------+--------+------------+----------------+--------+-----------+
|        1720|EDUARDO NANTES BO...|   EDUARDO BOLSONARO|   ELEITO POR QP|                        2179|             16124|                  13.51|       2018|       6|       61018|               2|       0|          0|
|        1011|ALEXANDRE ARAUJO ...|ALEXANDRE DAUAGE ...|        SUPLENTE|                           0|             16124|       

In [12]:
particoes = df_total_votos_candidato.select('PARTICAO_ID').distinct().collect()

In [13]:
particoes.sort(reverse=True)

In [14]:
len(particoes)

4390

In [15]:
for chave, particao in enumerate(particoes):
    particao_id = particao['PARTICAO_ID']
    print(chave,particao_id)
    particao_df = df_total_votos_candidato.filter(df_total_votos_candidato['PARTICAO_ID'] == particao_id).drop('PARTICAO_ID', 'ID_LINHA')
    particao_pandas = particao_df.toPandas()
    nome_arquivo = f'../data/processed/total_votos_candidato/df_total_votos_candidatos_{particao_id}.csv'
    particao_pandas.to_csv(nome_arquivo, header=True, sep='|')

0 60130156


1 60130155


2 60130154


3 60130153


4 60130152


...

3937 452
3938 451
3939 450
3940 449
3941 448
3942 447
3943 446
3944 445
3945 444
3946 443
3947 442
3948 441
3949 440
3950 439
3951 438
3952 437
3953 436
3954 435
3955 434
3956 433
3957 432
3958 431
3959 430
3960 429
3961 428
3962 427
3963 426
3964 425
3965 424
3966 423
3967 422
3968 421
3969 420
3970 419
3971 418
3972 417
3973 416
3974 415
3975 414
3976 413
3977 412
3978 411
3979 410
3980 409
3981 408
3982 407
3983 406
3984 405
3985 404
3986 403
3987 402
3988 401
3989 400
3990 399
3991 398
3992 397
3993 396
3994 395
3995 394
3996 393
3997 392
3998 391
3999 390
4000 389
4001 388
4002 387
4003 386
4004 385
4005 384
4006 383
4007 382
4008 381
4009 380
4010 379
4011 378
4012 377
4013 376
4014 375
4015 374
4016 373
4017 372
4018 371
4019 370
4020 369
4021 368
4022 367
4023 366
4024 365
4025 364
4026 363
4027 362
4028 361
4029 360
4030 359
4031 358
4032 357
4033 356
4034 355
4035 354
4036 353
4037 352
4038 351
4039 350
4040 349
4041 348
4042 347
4043 346
4044 345
4045 344
4046 343
4047 342
4

4244 145
4245 144
4246 143
4247 142
4248 141
4249 140
4250 139
4251 138
4252 137
4253 136
4254 135
4255 134
4256 133
4257 132
4258 131
4259 130
4260 129
4261 128
4262 127
4263 126


4264 125
4265 124
4266 123
4267 122
4268 121
4269 120
4270 119
4271 118
4272 117
4273 116
4274 115
4275 114
4276 113
4277 112
4278 111
4279 110
4280 109
4281 108
4282 107
4283 106
4284 105
4285 104
4286 103
4287 102
4288 101
4289 100
4290 99
4291 98
4292 97
4293 96
4294 95
4295 94
4296 93
4297 92
4298 91
4299 90
4300 89
4301 88
4302 87
4303 86
4304 85
4305 84
4306 83
4307 82
4308 81
4309 80
4310 79
4311 78
4312 77
4313 76
4314 75
4315 74
4316 73
4317 72
4318 71
4319 70
4320 69
4321 68
4322 67
4323 66
4324 65
4325 64
4326 63
4327 62
4328 61
4329 60
4330 59
4331 58
4332 57
4333 56
4334 55
4335 54
4336 53
4337 52
4338 51
4339 50
4340 49
4341 48
4342 47
4343 46
4344 45
4345 44
4346 43


4347 42
4348 41
4349 40
4350 39
4351 38
4352 37
4353 36
4354 35
4355 34
4356 33
4357 32
4358 31
4359 30
4360 29
4361 28
4362 27
4363 26
4364 25
4365 24
4366 23
4367 22
4368 21


4369 20
4370 19
4371 18
4372 17
4373 16
4374 15
4375 14
4376 13
4377 12
4378 11
4379 10
4380 9
4381 8
4382 7
4383 6
4384 5
4385 4
4386 3
4387 2
4388 1
4389 0


In [16]:
spark.stop()

- Total Votos Partido

In [40]:
df_total_votos_partido = spark \
    .read \
    .parquet('../data/processed/total_votos_partido.parquet')
df_total_votos_partido.show()

+-----------+--------+------------+----------+-------------+--------------------+----------------+----------------+-------------------+--------------------------+-----------------------+
|ANO_ELEICAO|CD_CARGO|CD_MUNICIPIO|NR_PARTIDO|   SG_PARTIDO|          NM_PARTIDO|CD_SIT_TOT_TURNO|DS_SIT_TOT_TURNO|TOTAL_VOTOS_PARTIDO|TOTAL_VOTOS_PARTIDO_CIDADE|PERCENTUAL_VOTOS_CIDADE|
+-----------+--------+------------+----------+-------------+--------------------+----------------+----------------+-------------------+--------------------------+-----------------------+
|       2018|       6|       61042|        17|          PSL|Partido Social Li...|               5|        SUPLENTE|                 43|                      7319|                   0.59|
|       2018|       6|       61042|        36|          PTC|Partido Trabalhis...|               4|      NÃO ELEITO|                  0|                      7319|                    0.0|
|       2018|       6|       61042|        13|           PT|Parti

In [41]:
df_total_votos_partido = df_total_votos_partido.select(
    'NM_PARTIDO',
    'TOTAL_VOTOS_PARTIDO',
    'TOTAL_VOTOS_PARTIDO_CIDADE',
    'ANO_ELEICAO',
    'CD_CARGO',
    'CD_MUNICIPIO',
    'NR_PARTIDO',
    'SG_PARTIDO'
)

In [42]:
numero_linha_particao = 1000
df_total_votos_partido = df_total_votos_partido.withColumn('ID_LINHA', monotonically_increasing_id()) 
df_total_votos_partido = df_total_votos_partido.withColumn('PARTICAO_ID', (df_total_votos_partido['ID_LINHA'] / numero_linha_particao).cast('integer'))
df_total_votos_partido.show()

+--------------------+-------------------+--------------------------+-----------+--------+------------+----------+-------------+--------+-----------+
|          NM_PARTIDO|TOTAL_VOTOS_PARTIDO|TOTAL_VOTOS_PARTIDO_CIDADE|ANO_ELEICAO|CD_CARGO|CD_MUNICIPIO|NR_PARTIDO|   SG_PARTIDO|ID_LINHA|PARTICAO_ID|
+--------------------+-------------------+--------------------------+-----------+--------+------------+----------+-------------+--------+-----------+
|Partido Social Li...|                 43|                      7319|       2018|       6|       61042|        17|          PSL|       0|          0|
|Partido Trabalhis...|                  0|                      7319|       2018|       6|       61042|        36|          PTC|       1|          0|
|Partido dos Traba...|                  1|                      7319|       2018|       6|       61042|        13|           PT|       2|          0|
|Partido Socialist...|                 60|                      7319|       2018|       6|       610

In [43]:
particoes = df_total_votos_partido.select('PARTICAO_ID').distinct().collect()

In [44]:
for chave, particao in enumerate(particoes):
    particao_id = particao['PARTICAO_ID']
    print(chave,particao_id)
    particao_df = df_total_votos_partido.filter(df_total_votos_partido['PARTICAO_ID'] == particao_id).drop('PARTICAO_ID', 'ID_LINHA')
    particao_pandas = particao_df.toPandas()
    nome_arquivo = f'../data/processed/total_votos_partido/df_total_votos_partido_{particao_id}.csv'
    particao_pandas.to_csv(nome_arquivo, header=True, sep='|')

0 31
1 34
2 28
3 26
4 27
5 12
6 22
7 1
8 13
9 6
10 16
11 3
12 20
13 40
14 5
15 19
16 15
17 37
18 9
19 17
20 35
21 4
22 8
23 23
24 39
25 7
26 10
27 38
28 25
29 24
30 29
31 21
32 32
33 11
34 33
35 14
36 2
37 30
38 0
39 18
40 36
41 8589941
42 8589951
43 8589948
44 8589954
45 8589953
46 8589937
47 8589949
48 8589956
49 8589952
50 8589938
51 8589947
52 8589944
53 8589959
54 8589940
55 8589942
56 8589950
57 8589936
58 8589935
59 8589955
60 8589943
61 8589958
62 8589957
63 8589960
64 8589946
65 8589934
66 8589939
67 8589945
68 17179894
69 17179890
70 17179885
71 17179882
72 17179878
73 17179874
74 17179891
75 17179892
76 17179889
77 17179893
78 17179876
79 17179884
80 17179879
81 17179869
82 17179888
83 17179873
84 17179880
85 17179872
86 17179881
87 17179870
88 17179886
89 17179871
90 17179875
91 17179887
92 17179883
93 17179877
94 25769818
95 25769809
96 25769810
97 25769821
98 25769822
99 25769815
100 25769828
101 25769813
102 25769808
103 25769827
104 25769804
105 25769803
106 25769825
10

- Desempenho Partido

In [45]:
df_desempenho_candidatos = spark \
    .read \
    .parquet('../data/processed/desempenho_candidatos.parquet')
df_desempenho_candidatos.show()

+------------+--------------------+--------------------+----------------+----------------+
|CD_MUNICIPIO|        NM_CANDIDATO|   NM_URNA_CANDIDATO|TOTAL_VOTOS_2018|TOTAL_VOTOS_2022|
+------------+--------------------+--------------------+----------------+----------------+
|       61000|   ABDULBASET JAROUR|        ABDUL JAROUR|               0|               0|
|       61000|ABRAHAM BRAGANÇA ...|   ABRAHAM WEINTRAUB|               0|               0|
|       61000|ADELIA DE JESUS S...|  DRA. ADÉLIA SOARES|               0|               0|
|       61000|ADENILSON PEREIRA...|    ADENILSON AMORIM|               0|               0|
|       61000|      ADEVAR CAMPANA|      ADEVAR CAMPANA|               0|               0|
|       61000| ADEZIO DIAS BARBOZA|         ADEZIO DIAS|               0|               0|
|       61000|ADILSON ALVES DE ...|ENG. ADILSON OLIV...|               0|               0|
|       61000|ADRIANA ALVES DE ...|    ADRIANA OLIVEIRA|               0|               0|

In [46]:
df_desempenho_candidatos = df_desempenho_candidatos.select(
    'NM_CANDIDATO',
    'NM_URNA_CANDIDATO',
    'TOTAL_VOTOS_2018',
    'TOTAL_VOTOS_2022',
    'CD_MUNICIPIO'


)

In [47]:
df_desempenho_candidatos = df_desempenho_candidatos.withColumn('ID_LINHA', monotonically_increasing_id()) 
numero_linha_particao = 1000
df_desempenho_candidatos = df_desempenho_candidatos.withColumn('PARTICAO_ID', (df_desempenho_candidatos['ID_LINHA'] / numero_linha_particao).cast('integer'))
df_desempenho_candidatos.show()

+--------------------+--------------------+----------------+----------------+------------+--------+-----------+
|        NM_CANDIDATO|   NM_URNA_CANDIDATO|TOTAL_VOTOS_2018|TOTAL_VOTOS_2022|CD_MUNICIPIO|ID_LINHA|PARTICAO_ID|
+--------------------+--------------------+----------------+----------------+------------+--------+-----------+
|   ABDULBASET JAROUR|        ABDUL JAROUR|               0|               0|       61000|       0|          0|
|ABRAHAM BRAGANÇA ...|   ABRAHAM WEINTRAUB|               0|               0|       61000|       1|          0|
|ADELIA DE JESUS S...|  DRA. ADÉLIA SOARES|               0|               0|       61000|       2|          0|
|ADENILSON PEREIRA...|    ADENILSON AMORIM|               0|               0|       61000|       3|          0|
|      ADEVAR CAMPANA|      ADEVAR CAMPANA|               0|               0|       61000|       4|          0|
| ADEZIO DIAS BARBOZA|         ADEZIO DIAS|               0|               0|       61000|       5|     

In [48]:
particoes = df_desempenho_candidatos.select('PARTICAO_ID').distinct().collect()

In [50]:
len(particoes)

4146

In [51]:
for chave, particao in enumerate(particoes):
    particao_id = particao['PARTICAO_ID']
    print(chave,particao_id)
    particao_df = df_desempenho_candidatos.filter(df_desempenho_candidatos['PARTICAO_ID'] == particao_id).drop('PARTICAO_ID', 'ID_LINHA')
    particao_pandas = particao_df.toPandas()
    nome_arquivo = f'../data/processed/desempenho_candidato/df_desempenho_candidato_{particao_id}.csv'
    particao_pandas.to_csv(nome_arquivo, header=True, sep='|')

0 148


1 463
2 471
3 496
4 243
5 392
6 31
7 516
8 85
9 137
10 251
11 451
12 65
13 458
14 53
15 255
16 481
17 133
18 296
19 472
20 78
21 322
22 513
23 321
24 362
25 375
26 108
27 155
28 34
29 193
30 211
31 368
32 101
33 115
34 126
35 81
36 385
37 28
38 183
39 210
40 436
41 497
42 300
43 406
44 412
45 76
46 26
47 27
48 332
49 501
50 384
51 44
52 159
53 192
54 271
55 253
56 103
57 236
58 329
59 460
60 12
61 336
62 350
63 223
64 388
65 417
66 91
67 409
68 333
69 222
70 285
71 22
72 128
73 209
74 330
75 372
76 122
77 230
78 319
79 493
80 93
81 157
82 190
83 225
84 232
85 233
86 246
87 346
88 360
89 367
90 476
91 111
92 224
93 47
94 140
95 177
96 416
97 132
98 152
99 444
100 185
101 305
102 353
103 355
104 291
105 325
106 386
107 435
108 473
109 146
110 259
111 512
112 1
113 206
114 52
115 297
116 363
117 212
118 274
119 182
120 442
121 13
122 218
123 280
124 348
125 440
126 6
127 16
128 86
129 474
130 168
131 205
132 3
133 142
134 178
135 308
136 328
137 20
138 40
139 164
140 429
141 470
142 169
1

491 215
492 410
493 172
494 252
495 399
496 144
497 153
498 420
499 0
500 18
501 475
502 74
503 138
504 104
505 180
506 468
507 483
508 134
509 407
510 36
511 187
512 208
513 315
514 393
515 89
516 401
517 422
518 517
519 8590266
520 8590371
521 8589941
522 8590381
523 8590033
524 8590194
525 8590310
526 8589951
527 8590369
528 8590031
529 8590119
530 8590157
531 8590199
532 8590252
533 8590280
534 8590045
535 8590355
536 8590437
537 8589948
538 8590255
539 8590410
540 8590282
541 8590414
542 8590047
543 8590068
544 8590073
545 8590319
546 8590091
547 8590260
548 8590296
549 8590328
550 8590331
551 8590048
552 8590184
553 8590307
554 8590177
555 8590233
556 8590295
557 8590357
558 8589974
559 8589980
560 8590013
561 8590025
562 8590055
563 8590232
564 8590274
565 8589968
566 8589997
567 8590380
568 8590398
569 8589954
570 8590123
571 8590287
572 8590386
573 8590142
574 8590245
575 8590253
576 8589953
577 8589970
578 8590075
579 8590215
580 8589981
581 8590230
582 8590432
583 8590449
58

790 8590401
791 8590311
792 8590334
793 8590201
794 8590429
795 8589992
796 8590211
797 8590377
798 8590042
799 8589957
800 8590140
801 8590190
802 8590273
803 8590305
804 8590053
805 8590078
806 8590089
807 8590092
808 8590407
809 8589965
810 8590191
811 8590442
812 8590012
813 8590262
814 8590436
815 8590035
816 8590246
817 8590426
818 8590122
819 8590397
820 8590404
821 8590412
822 8590090
823 8590170
824 8590235
825 8590264
826 8590269
827 8590283
828 8589987
829 8590101
830 8590289
831 8589960
832 8589972
833 8590102
834 8590125
835 8590064
836 8590096
837 8590419
838 8590038
839 8590067
840 8590171
841 8590182
842 8590183
843 8590361
844 8590403
845 8589946
846 8590141
847 8590379
848 8590409
849 8590153
850 8590325
851 8590354
852 8590275
853 8590314
854 8590360
855 8589985
856 8590071
857 8590175
858 8590227
859 8590086
860 8590105
861 8590107
862 8590139
863 8590143
864 8590330
865 8590144
866 8590154
867 8590365
868 8590448
869 8590212
870 8590388
871 8590206
872 8590277
873 

1048 17180286
1049 17180228
1050 17180127
1051 17180054
1052 17180062
1053 17179899
1054 17180198
1055 17180222
1056 17179955
1057 17179890


...

3244 51540044
3245 51539821
3246 51539994
3247 51540009
3248 51540072
3249 51540102
3250 51539676
3251 51539795
3252 51539816
3253 51539900
3254 51539645
3255 51539773
3256 51540023
3257 51539828
3258 51539868
3259 51539738
3260 51539966
3261 51539980
3262 51540034
3263 51539830
3264 51540046
3265 51540100
3266 51539613
3267 51539627
3268 51539632
3269 51539654
3270 51539872
3271 51539939
3272 51540060
3273 51539674
3274 51539700
3275 51539881
3276 51539895
3277 51540055
3278 51539623
3279 51539655
3280 51539658
3281 51539973
3282 51539771
3283 51539896
3284 51539911
3285 51540000
3286 51540096
3287 51539697
3288 51539918
3289 51540020
3290 51540101
3291 51539792
3292 51539904
3293 51539989
3294 51539814
3295 51539826
3296 51539622
3297 51539906
3298 51539617
3299 51539867
3300 51539866
3301 51539675
3302 51540062
3303 51539647
3304 51539687
3305 51539860
3306 51540103
3307 51539689
3308 51540018
3309 51540065
3310 51539652
3311 51539767
3312 51539787
3313 51540082
3314 51540007
3315 5

3595 60129970
3596 60130046
3597 60130075
3598 60130111
3599 60129859
3600 60129884
3601 60129753
3602 60130011
3603 60129927
3604 60129940
3605 60129981
3606 60129677
3607 60129759
3608 60129602
3609 60129678
3610 60129761
3611 60129573
3612 60129779
3613 60129591
3614 60129728
3615 60129920
3616 60129544
3617 60129603
3618 60129628
3619 60129710
3620 60130018
3621 60129772
3622 60129989
3623 60130053
3624 60130115
3625 60129701
3626 60130029
3627 60130024
3628 60129647
3629 60130077
3630 60129685
3631 60129813
3632 60129912
3633 60129808
3634 60129839
3635 60129938
3636 60130030
3637 60129776
3638 60130073
3639 60129545
3640 60130091
3641 60129604
3642 60129823
3643 60130019
3644 60130064
3645 60129688
3646 60129806
3647 60129992
3648 60130010
3649 60130120
3650 60129665
3651 60129770
3652 60129937
3653 60129659
3654 60129674
3655 60129850
3656 60129672
3657 60129794
3658 60129809
3659 60129883
3660 60129886
3661 60130082
3662 60129769
3663 60129775
3664 60130071
3665 60130105
3666 6

3727 60129971
3728 60129606
3729 60129625
3730 60129718
3731 60129851
3732 60129854
3733 60130103
3734 60129897
3735 60129973
3736 60129581
3737 60129723
3738 60129773
3739 60129787
3740 60129892
3741 60129903
3742 60129915
3743 60130067
3744 60130107
3745 60129616
3746 60129834
3747 60129846
3748 60129875
3749 60129702
3750 60129946
3751 60129972
3752 60129745
3753 60129821
3754 60130034
3755 60129715
3756 60129790
3757 60130033
3758 60130036
3759 60129751
3760 60130031
3761 60129563
3762 60129666
3763 60129957
3764 60129829
3765 60129906
3766 60129874
3767 60129853
3768 60129879
3769 60129675
3770 60129917
3771 60129978
3772 60129746
3773 60129587
3774 60129607
3775 60129660
3776 60129804
3777 60129893
3778 60130094
3779 60129542
3780 60129747
3781 60129961
3782 60129550
3783 60129559
3784 60129708
3785 60129774
3786 60130037
3787 60129671
3788 60129734
3789 60129814
3790 60130022
3791 60129717
3792 60129731
3793 60129818
3794 60129914
3795 60129754
3796 60129771
3797 60129781
3798 6

3987 60130121
3988 60129705
3989 60129739
3990 60129949
3991 60129546
3992 60129636
3993 60129877
3994 60130113
3995 60129621
3996 60129630
3997 60130038
3998 60129985
3999 60129999
4000 60130015
4001 60129852
4002 60129918
4003 60130055
4004 60129907
4005 60129564
4006 60129832
4007 60129910
4008 60129952
4009 60129576
4010 60129668
4011 60129707
4012 60129730
4013 60129756
4014 60129812
4015 60129858
4016 60129962
4017 60130080
4018 60130085
4019 60129777
4020 60129786
4021 60130039
4022 60129582
4023 60129619
4024 60129865
4025 60129998
4026 60129891
4027 60130002
4028 60129727
4029 60129692
4030 60129900
4031 60130095
4032 60130035
4033 60129595
4034 60129706
4035 60129601
4036 60129633
4037 60129650
4038 60130013
4039 60129654
4040 60129788
4041 60129939
4042 60129861
4043 60130065
4044 60129558
4045 60129622
4046 60129785
4047 60129609
4048 60129571
4049 60129694
4050 60129842
4051 60129984
4052 60129598
4053 60129798


4054 60129882
4055 60130087
4056 60129568
4057 60129816
4058 60129873
4059 60129926
4060 60129627
4061 60129724
4062 60129868
4063 60129958
4064 60129817
4065 60129579
4066 60129831
4067 60129855
4068 60129584
4069 60129869
4070 60129919
4071 60129631
4072 60130089
4073 60129566
4074 60129637
4075 60129696
4076 60129698
4077 60129593
4078 60129810
4079 60129881
4080 60130008
4081 60130097
4082 60129567
4083 60129638
4084 60129642
4085 60129695
4086 60129836
4087 60129597
4088 60129657
4089 60129991
4090 60130005
4091 60130081
4092 60129655
4093 60129801
4094 60130012
4095 60130016
4096 60129894
4097 60129663
4098 60129967
4099 60129913
4100 60129664
4101 60129681
4102 60129662
4103 60129895
4104 60129557
4105 60129765
4106 60129783
4107 60129899
4108 60129871
4109 60129969
4110 60130025
4111 60129588
4112 60129693
4113 60130051
4114 60130020
4115 60129735
4116 60129590
4117 60129872
4118 60129592
4119 60129757
4120 60129737
4121 60129553
4122 60130007
4123 60130093
4124 60129826
4125 6

In [52]:
spark.stop()